In this notebook, we are going to go through the process of understanding fully the code that is being presented to us. 

In [35]:
import itertools
import numpy as np
import cvxpy as cp
import pandas as pd
from enum import Enum
from typing import TypeVar
from pydantic import BaseModel
from tqdm import tqdm
from datetime import datetime

CENTER_NODE = "CENTAURI"  # Name of center Node
ORIGIN_TOKEN = "WETH"
OBJ_TOKEN = "ATOM"

MAX_RESERVE = 1e10

TAssetId = TypeVar("TAssetId")
TNetworkId = TypeVar("TNetworkId")

def populate_chain_dict(chains: dict[TNetworkId, list[TAssetId]], center_node: TNetworkId):
    # Add tokens with denom to Center Node
    # Basic IBC transfer
    for chain, tokens in chains.items():
        if chain != center_node:
            chains[center_node].extend(f"{chain}/{token}" for token in tokens)

    1# Add tokens from Center Node to outers
    
    # Simulate IBC transfer through Composable Cosmos
    for chain, tokens in chains.items():
        if chain != center_node:
            chains[chain].extend(
                f"{center_node}/{token}"
                for token in chains[center_node]
                if f"{chain}/" not in token
            )

# This function takes in your tokens, market makers, and reserves and returns the optimal trade. 
# You get the deltas, lambdas, psi, and etas           
def solve(
    all_tokens: list[TAssetId],
    all_cfmms: list[tuple[TAssetId, TAssetId]],
    reserves: list[np.ndarray[np.float64]],
    cfmm_tx_cost: list[float],
    fees: list[float],
    ibc_pools: int,
    origin_token: TAssetId,
    number_of_init_tokens: float,
    obj_token: TAssetId,
    force_eta: list[float] = None,
):
    
    # Build local-global matrices
    count_tokens = len(all_tokens)
    count_cfmms = len(all_cfmms)

    current_assets = np.zeros(count_tokens)
    current_assets[all_tokens.index(origin_token)] = number_of_init_tokens

    A = []
    for cfmm in all_cfmms:
        n_i = len(cfmm)
        A_i = np.zeros((count_tokens, n_i))
        for i, token in enumerate(cfmm):
            A_i[all_tokens.index(token), i] = 1
        A.append(A_i)

    # Build variables
    
    # tendered (given) amount
    deltas = [cp.Variable(len(l), nonneg=True) for l in all_cfmms]
    
    # received (wanted) amounts
    lambdas = [cp.Variable(len(l), nonneg=True) for l in all_cfmms]
    eta = cp.Variable(
        count_cfmms, nonneg=True
    )  # Binary value, indicates tx or not for given pool

    # network trade vector - net amount received over all trades(transfers/exchanges)
    psi = cp.sum([A_i @ (LAMBDA - DELTA) for A_i, DELTA, LAMBDA in zip(A, deltas, lambdas)])
    
    # Objective is to trade number_of_init_tokens of asset origin_token for a maximum amount of asset objective_token
    obj = cp.Maximize(psi[all_tokens.index(obj_token)] - eta @ cfmm_tx_cost)

    # Reserves after trade
    new_reserves = [
        R + gamma_i * D - L for R, gamma_i, D, L in zip(reserves, fees, deltas, lambdas)
    ]

    # Trading function constraints
    constrains = [
        psi + current_assets >= 0,
    ]

    # Pool constraint (Uniswap v2 like)
    for i in range(count_cfmms - ibc_pools):
        constrains.append(cp.geo_mean(new_reserves[i]) >= cp.geo_mean(reserves[i]))

    # Pool constraint for IBC transfer (constant sum)
    # NOTE: Ibc pools are at the bottom of the cfmm list
    for i in range(count_cfmms - ibc_pools, count_cfmms):
        constrains.append(cp.sum(new_reserves[i]) >= cp.sum(reserves[i]))
        constrains.append(new_reserves[i] >= 0)

    # Enforce deltas depending on pass or not pass variable
    # MAX_RESERVE should be big enough so delta <<< MAX_RESERVE
    for i in range(count_cfmms):
        constrains.append(deltas[i] <= eta[i] * MAX_RESERVE)
        if force_eta:
            constrains.append(eta[i] == force_eta[i])

    # Set up and solve problem
    prob = cp.Problem(obj, constrains)
    prob.solve(verbose=False, solver=cp.ECOS)
    
    amount_out = psi.value[all_tokens.index(obj_token)]
    amount_in = -psi.value[all_tokens.index(origin_token)]
    total_fees = eta.value @ cfmm_tx_cost
    
    return deltas, lambdas, psi, eta, amount_out, amount_in, total_fees


 # Build local-global matrices


def get_mapping_matrices(all_tokens, all_cfmms):
    count_tokens = len(all_tokens)
    count_cfmms = len(all_cfmms)
    A = []
    for cfmm in all_cfmms:
        n_i = len(cfmm)
        A_i = np.zeros((count_tokens, n_i))
        for i, token in enumerate(cfmm):
            A_i[all_tokens.index(token), i] = 1
        A.append(A_i)
        
    return A

### Environment set up

In this next section of code, we establish what the center node is and the set of constant function market makers that exist. There are two types of them: 
* Regular constant function market makers which help you to exchange one token for another 
* IBC pools which are constant sum market makers which represent the movement of one token into another one

One thing that we could do is operationalize this using objects and stuff. Make it easier to track what is going on in the system. 

In [2]:
CENTER_NODE = "CENTAURI"  # Name of center Node

ORIGIN_TOKEN = "WETH"
OBJ_TOKEN = "ATOM"

chains: dict[str, list[str]] = {
    "ETHEREUM": ["WETH", "USDC", "SHIBA"],
    CENTER_NODE: [],
    "OSMOSIS": ["ATOM","SCRT"],
}

populate_chain_dict(chains,CENTER_NODE)

all_tokens = []
all_cfmms = []
reserves = []
fees = []
cfmm_tx_cost = []
ibc_pools = 0
tol = 1e-4

In [3]:
for other_chain, other_tokens in chains.items():
    
    # We add the tokens to the list of tokens. These are tokens that exist on that chain
    all_tokens.extend(other_tokens)
    
    # We also add all market makers to the list of market makers using a combination of 2 tokens
    all_cfmms.extend(itertools.combinations(other_tokens, 2))
    
for cfmm in all_cfmms:
    
    # Adding a simulation of reserves and transaction costs for each
    reserves.append(np.random.uniform(9500, 10051, 2))
    cfmm_tx_cost.append(np.random.uniform(0, 20))


In [4]:
# Creating the IBC pools here which represent constant sum pools 
# These are not really "pools" but the bridges that you have to go through to get to the other chain
for token_on_center in chains[CENTER_NODE]:
        for other_chain, other_tokens in chains.items():
            if other_chain != CENTER_NODE:
                for other_token in other_tokens:
                    # Check wether the chain has the token in center, or the other way around
                    # Could cause problems if chainName == tokensName (for example OSMOSIS)
                    if other_token in token_on_center or token_on_center in other_token:
                        all_cfmms.append((token_on_center, other_token))
                        reserves.append(np.random.uniform(10000, 11000, 2))
                        cfmm_tx_cost.append(np.random.uniform(0, 20))
                        ibc_pools += 1
                        
# simulate random fees for all the CFMMs, including those that are IBC pools
fees.extend(np.random.uniform(0.97, 0.999) for _ in range(len(all_cfmms)))

### Solving the problem

We now are going to solve the optimal routing problem using python's cvxpy module. The key here is that this initial solving should give us some $\eta$ values. These $\eta$ values are our initial proposal and then we slide through each one turning them into 0s and 1s. 

In [8]:
input_amount = 2000
deltas, lambdas, psi, n, amount_out, amount_in, total_fees = solve(
    all_tokens,
    all_cfmms,
    reserves,
    cfmm_tx_cost,
    fees,
    ibc_pools,
    ORIGIN_TOKEN,
    input_amount,
    OBJ_TOKEN,
)

/home/stephonh/.local/lib/python3.10/site-packages/cvxpy/problems/problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [9]:
t_values = sorted(n.value)

In [10]:
# We slide through each of the t values and turn each eta value into 0 or 1 depending on this threshold
# For each t value, we collect the results of the solve function and store them in a dictionary
# Will go through each of these later to see which ones produce the best results

result = {}

for t in tqdm(t_values): 
    eta_values = [1 if n.value[i] >= t else 0 for i in range(len(n.value))]
    
    # Trying to solve with these eta values
    try: 
        deltas, lambdas, psi, n_temp, amount_out, amount_in, total_fees = solve(
            all_tokens,
            all_cfmms,
            reserves,
            cfmm_tx_cost,
            fees,
            ibc_pools,
            ORIGIN_TOKEN,
            input_amount,
            OBJ_TOKEN,
            force_eta=eta_values,
        )
        
        result[t] = {
            "amount_in": amount_in,
            "amount_out": amount_out,
            "total_fees": total_fees,
            "eta": eta_values,
            'net': amount_out - total_fees,
            'deltas': deltas,
            'lambdas': lambdas,
            'psi': psi,
            'n': n_temp
        }
    except: 
        continue


  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:10<00:00,  3.84it/s]


In [11]:
# Get the index with the highest net value 
max_index = max(result, key=lambda x: result[x]['net'])

# Get the result with the highest net value
final_result = result[max_index]

In [14]:
print(f"Amount in: {final_result['amount_in']}")
print(f"Amount out: {final_result['amount_out']}")
print(f"Total fees: {final_result['total_fees']}")
print(f"Net: {final_result['net']}")

Amount in: 1999.9998434808351
Amount out: 1844.9689359303661
Total fees: 16.124586546744972
Net: 1828.8443493836212


We can see that the best result here gives us a net of 1828.84 tokens out after swapping 2000 tokens originally. We are going to dive into the swaps that are proposed from this. 

In [20]:
final_deltas = final_result['deltas']
final_lambdas = final_result['lambdas']
final_psi = final_result['psi'].value
final_eta = final_result['n'].value

In [23]:
# Getting indices of the etas that are 1
final_eta_indices = [i for i, x in enumerate(final_eta) if x == 1]

In [33]:
eta_thresh = 0.999999 
final_eta_indices = [i for i, x in enumerate(final_eta) if x >= eta_thresh]

In [34]:
final_eta_indices

[3, 15, 17, 21, 26, 27, 36, 39]

We have these final eta indices which represent the exchanges that are going to be used. We want to check that the network trade vector after using these all work out. Do these give the final amount of tokens we expect? 

In [36]:
token_mapping_matrices = get_mapping_matrices(all_tokens, all_cfmms)

In [41]:
psis = []

for index in final_eta_indices:
    mapping_matrix = token_mapping_matrices[index]
    delta = final_deltas[index].value
    lambda_ = final_lambdas[index].value
    psi = mapping_matrix @ (lambda_ - delta)
    psis.append(psi)

In [43]:
# Summing all the psis
final_psi = sum(psis)

In [51]:
final_psi 

array([-4.69644577e+02,  0.00000000e+00,  0.00000000e+00, -6.37233279e+02,
        4.53742687e+02,  0.00000000e+00, -1.12931497e+02, -1.49904013e+02,
        8.92634524e+02, -2.47806568e+02,  2.79663980e+02,  1.16921887e+02,
       -2.98856378e+02,  1.18441618e-02,  1.24965404e+02])

In [45]:
psis_v2 = [] 

# Going through all the indices now 
for index in range(len(final_eta)):
    mapping_matrix = token_mapping_matrices[index]
    delta = final_deltas[index].value
    lambda_ = final_lambdas[index].value
    psi = mapping_matrix @ (lambda_ - delta)
    psis_v2.append(psi)

In [46]:
final_psi_v2 = sum(psis_v2)

In [50]:
final_psi_v2

array([-1.99999984e+03,  1.45726170e-04,  1.45008955e-04,  1.40752716e-04,
        1.44731361e-04,  1.46784173e-04,  1.42892015e-04,  1.40814218e-04,
        1.39231760e-04,  1.43070684e-04,  1.84496894e+03,  1.38854047e-04,
        1.42551785e-04,  1.38530066e-04,  1.38766022e-04])

What we are seeing is that if you do the procedure of fixing the eta values to 0 or 1, the solver solution does not make sense. You are not finding a solution where you route all the tokens toward your destination. However, when you use all the cfmms regardless of the eta values they give you, you're getting the right solution. This is an issue and is probably because of numerical tolerance around things. We can't exactly just say that something is zero. 

We might need to do something where we do this on the fly. Instead of putting the eta values into the optimization, we may have to just not include those cfmms in it and create the solver problem on the fly. We can test that out and see how this goes. We will have to do a try except or something though given that we are not sure that all the swaps are possible and stuff. 


In [29]:
# def get_swap_instructions(mapping_matrices, i: int): 
#     psi_i = mapping_matrices[i] @ (lambdas[i].value - deltas[i].value)
    
#     # Get the indices where the values are not 0
#     indices = list(np.nonzero(psi_i)[0])
    
#     first_token = all_tokens[indices[0]]
#     second_token = all_tokens[indices[1]]
    
#     # Get the amount of tokens that are being traded
#     first_token_amount = psi_i[indices[0]]
#     second_token_amount = psi_i[indices[1]]
    
#     if first_token_amount < 0:
#         return f"Trade {-first_token_amount} {first_token} for {second_token_amount} {second_token}"
#     else:
#         return f"Trade {-second_token_amount} {second_token} for {first_token_amount} {first_token}"

In [31]:
# class Swap: 
    
#     def __init__(self, token_in: str, token_out: str, amount_in: float, amount_out: float):
#         self.token_in = token_in
#         self.token_out = token_out
#         self.amount_in = amount_in
#         self.amount_out = amount_out
        
#     def __str__(self) -> str:
#         return f"Trade {self.amount_in} {self.token_in} for {self.amount_out} {self.token_out}"
    
#     def __repr__(self) -> str:
#         return f'Swap("{self.token_in}", "{self.token_out}", {self.amount_in}, {self.amount_out})'
    
    

In [42]:
# def create_swap(i: int) -> Swap: 
#     psi_i = A[i] @ (deltas[i].value - lambdas[i].value)
    
#     # Get the indices where the values are not 0
#     indices = list(np.nonzero(psi_i)[0])
    
#     first_token = all_tokens[indices[0]]
#     second_token = all_tokens[indices[1]]
    
#     # Get the amount of tokens that are being traded
#     first_token_amount = psi_i[indices[0]]
#     second_token_amount = psi_i[indices[1]]
    
#     if first_token_amount > 0:
#         return Swap(first_token, second_token, first_token_amount, -second_token_amount)
#     else:
#         return Swap(second_token, first_token, second_token_amount, -first_token_amount)
    

In [81]:
# swaps = [] 
# for i in range(len(all_cfmms)):
#     swaps.append(create_swap(i))
    
# inventory = {token: 0 for token in all_tokens}
# inventory[ORIGIN_TOKEN] = input_amount
# inventories = []

# swaps_copy = swaps.copy()
# swaps_sorted = []

In [88]:
# start_token = 'WETH'

# # Get all swaps that start with the start_token
# start_swaps = [swap for swap in swaps_copy if swap.token_in == start_token]

# for s in start_swaps: 
#     inventory[s.token_in] -= s.amount_in
#     inventory[s.token_out] += s.amount_out
    
#     # Remove the swap from the list of swaps in swaps_copy
#     swaps_sorted.append(s)
#     swaps_copy.remove(s)
    
# inventories.append(inventory.copy())
    
# while len(swaps_copy) > 0:
#     for token, amount in inventory.items():
#         token_swaps = [swap for swap in swaps_copy if swap.token_in == token]
        
#         if len(token_swaps) == 0: 
#             continue
#         else: 
#             for s in token_swaps:
#                 inventory[s.token_in] -= s.amount_in
#                 inventory[s.token_out] += s.amount_out
                
#                 # Remove the swap from the list of swaps in swaps_copy
#                 swaps_sorted.append(s)
#                 swaps_copy.remove(s)
#     inventories.append(inventory.copy())

In [111]:
swaps_dict = {token: [] for token in all_tokens}

for swap in swaps_sorted:
    swaps_dict[swap.token_in].append(swap)

In [116]:
swaps_dict

{'WETH': [Swap("WETH", "USDC", 199.57826386917714, 195.550559018189),
  Swap("WETH", "SHIBA", 251.6344785814913, 247.7142051626943),
  Swap("WETH", "CENTAURI/OSMOSIS/ATOM", 310.64743764587143, 299.14934626284384),
  Swap("WETH", "CENTAURI/OSMOSIS/SCRT", 234.08856742037057, 228.9466891786847),
  Swap("WETH", "ETHEREUM/WETH", 1004.0512472145539, 995.255552042431)],
 'USDC': [Swap("USDC", "SHIBA", 0.006397628887808525, 0.006368674402247055),
  Swap("USDC", "CENTAURI/OSMOSIS/ATOM", 195.54302679085006, 193.84669972205623),
  Swap("USDC", "CENTAURI/OSMOSIS/SCRT", 4.123920584735185e-05, 1.370918617685853e-05),
  Swap("USDC", "ETHEREUM/USDC", 0.001088300625679467, 0.0010551002392591886)],
 'SHIBA': [Swap("SHIBA", "CENTAURI/OSMOSIS/ATOM", 131.25124555901945, 129.3198317585649),
  Swap("SHIBA", "CENTAURI/OSMOSIS/SCRT", 116.41882013976233, 117.21439989196746),
  Swap("SHIBA", "ETHEREUM/SHIBA", 0.05050308002317698, 0.04984245933993024)],
 'CENTAURI/OSMOSIS/ATOM': [Swap("CENTAURI/OSMOSIS/ATOM", "OS